In [1]:
import pandas as pd
import numpy as np
import sys

In [2]:
import os, shutil
list_rubrics = os.listdir('./NFAC2020_ReturnedRubrics_061820')

In [3]:
dataframe_pavol = pd.read_excel('./061820_NFAC2020_JudgingRecords.xlsx')
disqualified = dataframe_pavol[dataframe_pavol["Eligible?" ] == "DISQUALIFIED"]
disqualified_list = disqualified['trackingNumber'].to_list()
print(disqualified_list)


['SC013713FH', 'SC013794FH', 'SC013983FJ', 'SC013992FJ', 'SC014037FK', 'SC014064FK']


In [4]:
employee_type = dataframe_pavol.set_index('trackingNumber').to_dict()['Employee Type']

In [5]:
scores_df_original = pd.DataFrame()
scores_df = pd.DataFrame()

submitted_judges = {}

for rubric in list_rubrics:
    if rubric.endswith('.xlsx'):
        judge = rubric.split('_')[0]

        if judge == 'SC031452FE':
            judge = 'SC013452FE'
        submitted_judges[judge] = []

        df = pd.read_excel('./NFAC2020_ReturnedRubrics_061820/%s' % rubric, skiprows = 7, usecols='L:U')
        presenters = df.dropna(axis = 1, how='all').columns
        new_presenters = []
        for presenter in list(presenters):
            presenter_orig = presenter
            if 'S' in presenter: ###
                presenter = presenter.lstrip(presenter[0: presenter.find('S')]) ###
            if presenter.startswith('\xa0'):
                presenter = presenter.strip('\xa0')
            if presenter.startswith('SCO'): ###
                presenter = presenter.replace('SCO', 'SC0')
            if not presenter.startswith('SC0'):
                presenter = presenter.replace('SC', 'SC0')
            if not presenter.startswith('SC01'):
                presenter = presenter.replace('SC0', 'SC01')
            if not presenter[-2:].isupper():
                presenter = presenter.replace(presenter[-2:], presenter[-2:].upper())
            presenter = presenter[0:10]
            submitted_judges[judge].append(presenter)
            new_presenters.append(presenter)

        scores = df.dropna(axis = 1, how='all').iloc[[-1]].values
        scores_df_original = pd.concat([scores_df_original, pd.DataFrame(data = scores, columns = new_presenters, index=[judge])], axis = 0, sort = False)



In [6]:
scores_df = pd.DataFrame()

for column in scores_df_original.columns.to_list():
    if column not in disqualified_list and column.startswith('SC0'):
        dropped_column = scores_df_original[column].dropna().reset_index(drop=True, inplace=False)
        sorted_scores = dropped_column.sort_values(ascending=False).reset_index(drop=True, inplace=False)
        scores_df = pd.concat([scores_df, sorted_scores], axis =1)

In [7]:
judge_number = 3
mean = scores_df.iloc[:judge_number, :].mean()
scores_df.loc['mean'] = mean

In [8]:
for column in scores_df.columns.to_list():
    scores_df.loc['Employee Type', column] = employee_type[column]

In [9]:
new_scores_df = scores_df.loc[['Employee Type', 'mean', 3, 4]].transpose()
new_scores_df = new_scores_df.rename(columns={3: '4th score', 4: '5th score'})
postdoc_df = new_scores_df.loc[(new_scores_df['Employee Type'] == 'Post Doctoral Fellow') |\
                               (new_scores_df['Employee Type'] == 'Research Fellow')]
postdoc_df = postdoc_df.sort_values(['mean', '4th score', '5th score'], ascending=[False, False, False])

postbac_predoc_df = new_scores_df.loc[(new_scores_df['Employee Type'] != 'Post Doctoral Fellow') &\
                               (new_scores_df['Employee Type'] != 'Research Fellow')]
postbac_predoc_df = postbac_predoc_df.sort_values(['mean', '4th score', '5th score'], ascending=[False, False, False])

In [10]:
writer = pd.ExcelWriter('Scores.xlsx', engine='xlsxwriter')
postdoc_df.to_excel(writer, sheet_name = 'Scores', startcol=0)
postbac_predoc_df.to_excel(writer, sheet_name='Scores', startcol=7)
writer.save()

In [11]:
print('Fraction of postdoc fellows: ', round(len(postdoc_df)/len(new_scores_df)*8, 3))
print('Fraction of postbac, pre-doc fellows: ', round(len(postbac_predoc_df)/len(new_scores_df)*8, 3))

Fraction of postdoc fellows:  6.031
Fraction of postbac, pre-doc fellows:  1.969
